
# 🧑‍💻 Face and Gender Detection using OpenCV & Pre-trained Caffe Model

This project implements **real-time face and gender detection** using **OpenCV**, **Haar cascades** for face detection, and a **pre-trained Caffe deep learning model** for gender classification. The application works with a **webcam feed**.

---

## 📂 Components Used

1. **OpenCV Haar Cascade**

   * `haarcascade_frontalface_default.xml` is used for detecting faces in the video stream.

2. **Pre-trained Gender Detection Model (Caffe)**

   * `deploy_gender.prototxt` → Defines the model architecture.
   * `gender_net.caffemodel` → Contains the trained model weights.

3. **Gender Classes**

   * Two categories are predicted:

     * `"Male"`
     * `"Female"`

---

## ⚙️ Steps Performed in Code

### 1. Import Required Libraries

```python
import cv2
import numpy as np
```

### 2. Load Pre-trained Models

* **Face Detection Model** (Haar cascade).
* **Gender Detection Model** (`.prototxt` + `.caffemodel`).

### 3. Start Webcam Stream

```python
cap = cv2.VideoCapture(0)
```

### 4. Detect Faces in Each Frame

* Convert frame to **grayscale**.
* Use `detectMultiScale()` to find faces.
* Draw a rectangle around detected faces.

### 5. Preprocess Face for Gender Prediction

* Extract face **ROI** (Region of Interest).
* Convert it into a **blob** using `cv2.dnn.blobFromImage()`.
* Normalize values with mean subtraction (values specific to Caffe training).

### 6. Gender Prediction

* Forward pass the blob through the **gender detection network**.
* Take the class with **maximum probability**.
* Overlay predicted label on the video frame.

### 7. Display Results

* Show real-time video feed with bounding boxes and gender labels.
* Press **`q`** to quit.

---

## 🖼️ Output

* A webcam window titled **"Face and Gender Detection"**.
* Detected faces are highlighted with a **green rectangle**.
* Predicted **gender** (Male/Female) is displayed above each face.

---

## ✅ What We Have Done

* Integrated **face detection** with **deep learning gender classification**.
* Processed **real-time video frames** using webcam.
* Successfully applied **computer vision + deep learning** pipeline with OpenCV.


In [1]:
!pip install opencv-python numpy


In [1]:
import cv2
import numpy as np

# Load pre-trained face detector (Haar cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load pre-trained gender detection model
MODEL_PROTO = "deploy_gender.prototxt"   # Path to .prototxt file
MODEL_WEIGHTS = "gender_net.caffemodel"  # Path to .caffemodel file
gender_net = cv2.dnn.readNetFromCaffe(MODEL_PROTO, MODEL_WEIGHTS)

# Gender classes
GENDERS = ["Male", "Female"]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        # Draw face rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Extract face ROI
        face_img = frame[y:y+h, x:x+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227),
                                     (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

        # Predict gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = GENDERS[gender_preds[0].argmax()]

        # Show gender label
        cv2.putText(frame, gender, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (255, 0, 255), 2)

    # Display output
    cv2.imshow("Face and Gender Detection", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
